# How to set up a basic Kili project

In this tutorial, we'll learn how to set up a basic Kili project.

Here are the steps that we must follow:

1. [Installing and instantiating Kili](#-Installing-and-instantiating-Kili)
2. [Creating a basic Kili project](#-Creating-a-basic-Kili-project)
3. [Adding assets to project](#-Adding-assets-to-project)
4. [Adding users to project](#-Adding-users-to-project)

## Installing and instantiating Kili

First, let's install and import the required modules.

In [1]:
!pip install --upgrade kili
from kili.client import Kili
import getpass

     |████████████████████████████████| 150 kB 8.2 MB/s eta 0:00:01
  Attempting uninstall: kili
    Found existing installation: kili 2.124.3
    Uninstalling kili-2.124.3:
      Successfully uninstalled kili-2.124.3


Now, let's set up variables needed to create an instance of the Kili object.
We'll need your API key and Kili's API endpoint.
If you're unsure how to look up your API key, refer to https://docs.kili-technology.com/docs/creating-an-api-key.

In [3]:
KILI_API_KEY = getpass.getpass("Please enter your API key: ")

With variables set up, we can now create an instance of the Kili object.

In [4]:
kili = Kili(
    api_key=KILI_API_KEY,
    # api_endpoint="https://cloud.kili-technology.com/api/label/v2/graphql",
    # the line above can be uncommented and changed if you are working with an on-premise version of Kili
)

## Creating a basic Kili project

To create a Kili project, you must first set up its interface.
We'll create a simple image project with just one simple classification job and two categories: `OBJECT_A` and `OBJECT_B`.

To learn more about Kili project interfaces, refer to https://docs.kili-technology.com/docs/customizing-project-interface

In [5]:
interface = {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "content": {
                "categories": {"OBJECT_A": {"name": "Object A"}, "OBJECT_B": {"name": "Object B"}},
                "input": "radio",
            },
        }
    }
}

result = kili.create_project(
    title="Test Project",
    description="Project Description",
    input_type="IMAGE",
    json_interface=interface,
)

For further processing, we'll need to find out what our project ID is. We can easily retrieve it from the project creation response message.

In [6]:
project_id = result["id"]

Now, let's add some assets to be labeled. We'll use some free off-the-shelf examples from the Internet.

## Adding assets to project

In [7]:
# Image url
url1 = "https://images.unsplash.com/photo-1546069901-ba9599a7e63c?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1480&q=80"
url2 = "https://images.unsplash.com/photo-1565958011703-44f9829ba187?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1065&q=80"
url3 = "https://images.unsplash.com/photo-1609951651556-5334e2706168?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=987&q=80"

assets = kili.append_many_to_dataset(
    project_id=project_id,
    content_array=[url1, url2, url3],
    external_id_array=["image_1", "image_2", "image_3"],
)

100%|██████████| 3/3 [00:00<00:00,  6.21it/s]


## Adding users to project

Now we need to add users to our project. Before we do that, we have to add them to our organization. **Note that you have to be an org admin to be able to do that**.
For more info on roles in an organization, refer to https://docs.kili-technology.com/docs/user-roles-in-organization.

In [ ]:
firstname = "Jane"
lastname = "Doe"
email = "no.such.email@no.such.domain.com"
password = "12345"
organization_role = "USER"

response = kili.create_user(email, password, organization_role, firstname, lastname)

If you already have users in your organization, here's how you can easily access their IDs:
First, retrieve your organization ID:

In [9]:
kili.organizations()

100%|██████████| 1/1 [00:00<00:00,  2.06it/s]


[{'id': 'cjx0vz05w0tmq08390mrl76bd', 'name': 'Kili Technology'}]

Then, based on your org ID, retrieve the full list of org users, with their e-mails:

In [11]:
all_org_users = kili.users(organization_id="cjx0vz05w0tmq08390mrl76bd")

all_emails = [i["email"] for i in all_org_users]

100%|██████████| 393/393 [00:01<00:00, 323.80it/s]


We'll use the e-mail of the new user to add our new user to our project:

In [ ]:
kili.append_to_roles(project_id, "no.such.email@no.such.domain.com", role="LABELER")

## Cleanup

To clean up, we need to simply remove the project that we created.

In [ ]:
kili.delete_project(project_id)

## Summary

Done. We've successfully set up a Kili project, defined its interface, created a brand new user, and finally added our new user to the new project. Well done!